Linear Relations.
To start let's just make an implementation of linear subspaces.


Columns of vrep are generators / span the sub space / the range of the vrep
Rows of hrep are constraints of subspace / the nullspace of hrep

I was told I should use NullSpace / Range as the terms. I guess that makes sense.


In [1]:
using LinearAlgebra
using Catlab
using Catlab.Doctrines

In [2]:
#A*x = 0

abstract type LinSpace end # LinRep?

struct NullSpace{T} <: LinSpace 
   A :: AbstractArray{T}
end
struct RangeSpace{T} <: LinSpace 
   A :: AbstractArray{T}
end

# nullspace is already a function. It doesn't make sense
# nrep
# rrep
function nrep(x::NullSpace) 
    NullSpace(nullspace(x.A))
end

function rrep(x::RangeSpace) 
    x
end

function nrep(x::RangeSpace)
    NullSpace(nullspace(x.A')')
end

function rrep(x::NullSpace)
    x
end


# meet is the intersection of 2 linear subspaces
function meet(x::NullSpace, y::NullSpace)
    NullSpace([x.A ; y.A])
end

function meet(x :: LinSpace, y :: LinSpace)
     xh = nrep(x)
     yh = nrep(y)  
     meet(xh, yh)
end


# \wedge 
∧ = meet


function join(x::RangeSpace, y::RangeSpace)
    RangeSpace([x.A y.A])
end

function join(x :: LinSpace, y :: LinSpace)
    x = rrep(x)
    y = rrep(y)
    join(x,y)
end
    
# \vee
∨ = join


function subspace(r :: RangeSpace, p :: NullSpace)
    all(isapprox.(p.A * r.A , 0; atol=eps(Float64), rtol=0))
end


function subspace(r :: LinSpace, p :: LinSpace)
    p = nrep(p)
    r = rrep(r)
    subspace(r,p)
end

# \subseteq
function ⊆(x :: LinSpace, y :: LinSpace)
    subspace(x,y)  
end

function ==(x :: LinSpace, y :: LinSpace)
    (x ⊆ y) && (y ⊆ x)
end

function top(n)
   Range(Matrix(I, n, n)) 
end 

# \top
⊤ = top

function bottom(n)
    NullSpace(Matrix(I, n, n)) 
end

# \bot
⊥ = bottom

# projection functions
function Base.getindex(p::RangeSpace, i)
    RangeSpace( p.A[i, :] )
end  

function Base.getindex(p::NullSpace, i)
    rrep(p)[i]
end    

function Base.size(p::RangeSpace)
    size(p.A)[1]
end

function Base.size(p::NullSpace)
    size(p.A)[2]
end

In [10]:
RangeSpace(Matrix(I,3,3)) ⊆ RangeSpace(Matrix(I,3,3))

true

The stratagem to talk about affine subspaces is to use homogenous coordinates. The last dimension is used as a constant.



Wrapping this functionality with a relational interface.
We need to track the seperation between input and output dimensions

How do we do this such that it won't suck to implement and/or use?

One option is to hold the "input" and "output" pieces seperately

In [5]:


abstract type LinRel end
    
struct LinRelR <: LinRel
    a :: RangeSpace # pre slice out the different sections?
    b :: RangeSpace # 
end

struct LinRelN <: LinRel
    a :: NullSpace # pre slice out the different sections?
    b :: NullSpace # invaraint is the
end

In [8]:



# abstract type LinRel
# LinRelV <: LinRel
# LinRelH <: LinRel

function nrep(x :: LinRelR)
    LinRelN( nrep(x.a) , nrep(x.b) )
end

function nrep(x :: LinRelN)
    x
end

function rrep(x :: LinRelN)
    LinRelR( rrep(x.a) , rrep(x.b) )
end

function rrep(x :: LinRelR)
    x
end

# Replicate all the operations?
function compose(p :: LinRelN, q :: LinRelN)
    (cp, na)= size(p.a.A)
    (cp, nb)= size(p.b.A)
    (cq, nb)= size(q.a.A)
    (cq, nc)= size(q.b.A)
    s = NullSpace([ p.a           p.b  zeros(cp,nc) ;
                    zeros(cq, na) q.a  q.b          ])
    s2 = vrep(s)
    LinRelR(  s2[1:na] ,  s2[(na + nb + 1):end] )
end
function compose(p :: LinRel, q :: LinRel)
    p = nrep(p)
    q = nrep(q)
    compose( p, q )
end
#=
function compose(p :: LinRelV, q :: LinRelV)
    p = hrep(p)
    compose(p,q)
end

function compose(p :: LinRelH, q :: LinRelV)
    q = hrep(q)
    compose(p,q)
end

function compose(p :: LinRelV, q:: LinRelH)
    p = hrep(p)
    compose(p,q)
end


function Base.size( p :: LinRelH)
    (size(p.a), size(p.b))
end

function Base.size( p :: LinRelV)
    (size(p.a), size(p.b))
end
=#
function Base.size( p :: LinRel)
    (size(p.a), size(p.b))
end
    
function fuse(p :: LinRelR)
    RangeSpace( [ p.a ; p.b ]  )
end

function fuse(p :: LinRelN)
    NullSpace( [ p.a p.b ] )
end

function subset(p :: LinRel, q :: LinRel)
    subset(fuse(p), fuse(q))
end

converse(p :: LinRelN) = LinRelN(p.b, p.a)
converse(p :: LinRelR) = LinRelR(p.b, p.a)


function id(n)
    LinRelR(Matrix(I,n,n), Matrix(I,n,n))
end



    
    

id (generic function with 1 method)

Monoidal Product

complement of vector space is also something to do.


In [7]:
#surely this is somewhere?
function diagzero(a, b)
    ar, ac = size(a)
    br, bc = size(b)
    [a              zeros( ar, bc) ;
      zeros(br, ac )    b] , 
end

function dsum(p :: LinRelR, r :: LinRelR)
    LinRelR( diagzero(p.a, r.a), diagzero(p.b , r.b)) 
end

function dsum(p :: LinRelN, r :: LinRelN)
    LinRelN( diagzero(p.a, r.a), diagzero(p.b , r.b)) 
end


delete(n::Int) = LinRelV(zeros(n,0),zeros(0,0))
mcopy(n::Int) = LinRelV( [Matrix(I,n,n) Matrix(I,n,n) ], diagzero(Matrix(I,n,n), Matrix(I,n,n)))
# pair(p :: LinRelN, q :: LinRelN) = 
proj1(n,m) = LinRelV( RangeSpace([Matrix(I,m,m) ; zeros( n - m , m )  ])  ,  RangeSpace(Matrix(I,m,m)) )
proj2(n,m) = LinRelV( RangeSpace([zeros( n - m , m )  ; Matrix(I,m,m) ])  ,  RangeSpace(Matrix(I,m,m)) )

LoadError: syntax: unexpected "end"

In [25]:


@instance Category(Int, LinRel) begin
  dom(M :: LinRel) = size(M.a)
  codom(M :: LinRel) = size(M.b)

  id(m::Int) = LinRelR( RangeSpace(Matrix(I,m,m)), RangeSpace(Matrix(I,m,m)))
  compose(M::LinRel, N::LinRel) = compose(M, N)
end

In [27]:
@instance MonoidalCategory(Int, LinRel) begin
  otimes(A :: Int, B :: Int) = A + B
  otimes(f :: LinRel, g :: LinRel) = dsum(f,g)
  munit() = 0
end

ErrorException: Method dom(::LinRel) not implemented in MonoidalCategory instance

In [ ]:
"""
@signature MonoidalCategoryWithDiagonals(Ob,Hom) => CartesianCategory(Ob,Hom) begin
  pair(f::Hom(A,B), g::Hom(A,C))::Hom(A,otimes(B,C)) <= (A::Ob, B::Ob, C::Ob)
  proj1(A::Ob, B::Ob)::Hom(otimes(A,B),A)
  proj2(A::Ob, B::Ob)::Hom(otimes(A,B),B)
end

braid(A::Ob, B::Ob)::Hom(otimes(A,B),otimes(B,A))


@signature SymmetricMonoidalCategory(Ob,Hom) => MonoidalCategoryWithDiagonals(Ob,Hom) begin
  mcopy(A::Ob)::Hom(A,otimes(A,A))
  delete(A::Ob)::Hom(A,munit())
  
  # Unicode syntax
  Δ(A::Ob) = mcopy(A)
  ◇(A::Ob) = delete(A)
end



"""


In [13]:
codom( compose(id(3), id(3)) )

StackOverflowError: StackOverflowError:

In [20]:
zeros(10,0)

10×0 Array{Float64,2}

In [ ]:
# LinRel makes sense 
struct LinRel
   shape  # :: Tuple{Int,Int}  # shape is a tuple of indices. The size of rep should equal the sum of shape
   rep :: LinSpace
end

function meet(x :: LinRel, y :: LinRel)
    @assert x.shape == y.shape
    LinRel(x.shape, meet(x.rep, y.rep))
end

function join(x :: LinRel, y :: LinRel)
    @assert x.shape == y.shape
    LinRel(x.shape, join(x.rep, y.rep))
end

function Base.size(x :: LinRel)
    x.shape
end

function compose(x :: LinRel, y :: LinRel)
    @assert x.shape[2] == y.shape[1]
    p = hrep(x.rep).A
    pc = size()
    a = p[:, 1:x.shape[1]]
    b = p[:, (x.shape[1]+1):end]
    q = hrep(y.rep).A   
    c = p[:, 1:y.shape[1]]
    d = p[:, (y.shape[1] + 1):end]
    [ a b zeros(size(p)[1])  )   ]
    LinRel(x.shape, join(x.rep, y.rep))
end

In [19]:
struct RelDomain
    dom :: Array{Symbol,1}
end

struct LinRel
    dom :: RelDomain
    codom :: RelDomain  
    rep :: Union{VRep, HRep}
end

struct LinRel2
    dom :: RelDomain
    codom :: RelDomain  
    a :: VRep # pre slice out the different sections?
    b :: Vrep 
end


struct LinRelV
    a :: VRep # pre slice out the different sections?
    b :: Vrep 
end

struct LinRelH
    a :: HRep # pre slice out the different sections?
    b :: Hrep 
end

ErrorException: type Array has no field size

In [20]:
RelDomain2([:i, :v, :i, :v])

RelDomain2(Symbol[:i, :v, :i, :v])

Here's an idea. In ordinary numpy multindexes is about direct product
But we could use hierarchical indxing instead fro direct sum

[0, 0, 0 ,0:]

How would broadcasting work?

What would i even need to broadcast though

relation of shape (n,m,p, ..) has n+ m + p _ .. variables/



In [ ]:
struct VRep2{T,S}
    a :: AbstractArray{T}
    shape :: S
end

function reshape(x :: VRep2, newshape :: ?  )
    @assert sum(size(x)) == sum(newshape)
    VRep( x.A, newshape )
end

In [30]:
typeof(size(reshape([1 2 3 4], (1,2,2)) ) )

Tuple{Int64,Int64,Int64}

What if we made the product category with a symbol category